# Review: Improved Training of Wasserstein GANs

###### ***Author***

### 1. Contribution

- critic weight clipping의 문제점을 toy dataset을 통해서 보였다.
- gradient penalty를 제안해서 위의 문제를 해결했다.
- 다양한 GAN 구조에 대해 안정적인 학습을 증명하고, weight clipping에 대한 성능 향상, high quality image generation 등이 가능하다.

### 2. Background & Difficulties with weight constraints

#### A. Background

![Untitled](https://github.com/yehjin-00/ybigta-generative-model-study/blob/main/GAN/WGAN-GP/image/1.png)

![Eq 1](https://github.com/yehjin-00/ybigta-generative-model-study/blob/main/GAN/WGAN-GP/image/2.png)


![Eq 2](https://github.com/yehjin-00/ybigta-generative-model-study/blob/main/GAN/WGAN-GP/image/3.png)

Eq 2

![Eq3](https://github.com/yehjin-00/ybigta-generative-model-study/blob/main/GAN/WGAN-GP/image/4.png)

Eq3

WGAN 손실함수(Eq1)을 max로 만드는 f를 f*라고 하자. 이 f*는 Eq2를 통해서 구할 수 있다. 

$x$ ~$P_g$, $y$~$P_r$로 샘플링을 해서 $x$와 $y$를 보간한 직선 중 $x$와 $y$ 사이의 점 $x_t$가 있다고 할 때, $x_t=tx+(1-t)y$ 로 나타낼 수 있다. (0 ≤ $t$ ≤ 1)

이때, 어떠한 $x_t$이든 Eq3이 만족된다. (증명은 appendix)

![Untitled](https://github.com/yehjin-00/ybigta-generative-model-study/blob/main/GAN/WGAN-GP/image/5.png)

## B. Weight Clipping의 문제점

- weight clipping: 업데이트된 weight를 특정 구간으로 제한하는 것 → 1-Lipschitz를 만족시키기 위함
- weight clippping이 최적화에 문제를 발생시킨다. optimization이 성공했을때조차 critic이 pathological value surface를 가지는 경우도 있다.

![윗줄(weight clipping): 데이터 분포의 higher moments를 잡아내는 데 실패했다. // 밑줄(gradient panelty)](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/fbdd8f1b-b609-4e0e-8a81-0b80f813672b/Untitled.png)

윗줄(weight clipping): 데이터 분포의 higher moments를 잡아내는 데 실패했다. // 밑줄(gradient panelty)

![Swiss Roll dataset을 학습시키는 도중 weight clipping을 사용할 때, 발생하는 vanish / explode를 보여주고 gradient penalty를 사용 시 문제가 발생하지 않는 것을 보여주는 gradient norm](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/bd941676-a304-457c-b6a9-93c5cc43f392/Untitled.png)

Swiss Roll dataset을 학습시키는 도중 weight clipping을 사용할 때, 발생하는 vanish / explode를 보여주고 gradient penalty를 사용 시 문제가 발생하지 않는 것을 보여주는 gradient norm

![weight clipping은 가중치가 clipping boundary 근처로 몰리지만 gradient panelty는 그렇지 않는 것을 알 수 있다. ](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/d1df75be-dfc7-464d-899e-760b801ccd3b/Untitled.png)

weight clipping은 가중치가 clipping boundary 근처로 몰리지만 gradient panelty는 그렇지 않는 것을 알 수 있다.